# Business Understanding

## Project Overview
For this project, you will use exploratory data analysis to generate insights for a business stakeholder.

## Business problem: 
Your company now sees all the big companies creating original video content and they want to get in on the fun. They have decided to create a new movie studio, but they don’t know anything about creating movies. You are charged with exploring what types of films are currently doing the best at the box office. You must then translate those findings into actionable insights that the head of your company's new movie studio can use to help decide what type of films to create.
## Project objectives: 
### Main Objective
To analyze movie data and uncover patterns in sales, popularity, ratings, and director influence across genres, providing actionable insights for business growth and strategy.

### Specific Objectives
1. **Genre by Sales**  
   Identify which genres generate the most revenue and analyze trends contributing to their sales performance.
   Tables:`bom.movie_gross.csv` , `rt.movie_info.tsv`

2. **Genre by Popularity**  
   Understand which genres are most popular among audiences and explore factors driving their popularity.
   Tables:`bom.movie_gross.csv`, `tmdb.movies.csv`

3. **Genre by Rating**  
   Examine the ratings of movies across different genres to evaluate their critical reception.
    DB:`im.db` Tables: `movie_basics`, `movie_rating`

4. **Directors by Genre**  
   Determine which directors are most associated with specific genres and assess their impact on genre success.
    DB:`im.db` Tables: `movie_basics`, `directors`

### The Data
In the folder `zippedData` are movie datasets from:

* [Box Office Mojo](https://www.boxofficemojo.com/)
* [IMDB](https://www.imdb.com/)
* [Rotten Tomatoes](https://www.rottentomatoes.com/)
* [TheMovieDB](https://www.themoviedb.org/)
* [The Numbers](https://www.the-numbers.com/)

Because it was collected from various locations, the different files have different formats. Some are compressed CSV (comma-separated values) or TSV (tab-separated values) files that can be opened using spreadsheet software or `pd.read_csv`, while the data from IMDB is located in a SQLite database.

![movie data erd](https://raw.githubusercontent.com/learn-co-curriculum/dsc-phase-2-project-v3/main/movie_data_erd.jpeg)

Note that the above diagram shows ONLY the IMDB data. You will need to look carefully at the features to figure out how the IMDB data relates to the other provided data files.

It is up to you to decide what data from this to use and how to use it. If you want to make this more challenging, you can scrape websites or make API calls to get additional data. If you are feeling overwhelmed or behind, we recommend you use only the following data files:

* `im.db.zip`
  * Zipped SQLite database (you will need to unzip then query using SQLite)
  * `movie_basics` and `movie_ratings` tables are most relevant
* `bom.movie_gross.csv.gz`
  * Compressed CSV file (you can open without expanding the file using `pd.read_csv`)

# Data Understanding 

In [1]:
#importing libraries for data manipulation (pandas, numpy) and visualization (seaborn, matplotlib)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import sqlite3
import warnings

# Suppress all warnings
warnings.filterwarnings('ignore')

In [2]:
# set the maximum number of columns to 40 to display all columns
pd.set_option('display.max_columns', 40)

<b>rt.movie_info.tsv</b>

In [3]:
movie_df = pd.read_csv('rt.movie_info.tsv', sep='\t')
movie_df.head()

,id,synopsis,rating,genre,director,writer,theater_date,dvd_date,currency,box_office,runtime,studio
0,1,"This gritty, fast-paced, and innovative police...",R,Action and Adventure|Classics|Drama,William Friedkin,Ernest Tidyman,"Oct 9, 1971","Sep 25, 2001",NaN,NaN,104 minutes,NaN
1,3,"New York City, not-too-distant-future: Eric Pa...",R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,"Aug 17, 2012","Jan 1, 2013",$,"600,000",108 minutes,Entertainment One
2,5,Illeana Douglas delivers a superb performance ...,R,Drama|Musical and Performing Arts,Allison Anders,Allison Anders,"Sep 13, 1996","Apr 18, 2000",NaN,NaN,116 minutes,NaN
3,6,Michael Douglas runs afoul of a treacherous su...,R,Drama|Mystery and Suspense,Barry Levinson,Paul Attanasio|Michael Crichton,"Dec 9, 1994","Aug 27, 1997",NaN,NaN,128 minutes,NaN
4,7,NaN,NR,Drama|Romance,Rodney Bennett,Giles Cooper,NaN,NaN,NaN,NaN,200 minutes,NaN


In [4]:
movie_df.tail()

,id,synopsis,rating,genre,director,writer,theater_date,dvd_date,currency,box_office,runtime,studio
1555,1996,Forget terrorists or hijackers -- there's a ha...,R,Action and Adventure|Horror|Mystery and Suspense,NaN,NaN,"Aug 18, 2006","Jan 2, 2007",$,"33,886,034",106 minutes,New Line Cinema
1556,1997,The popular Saturday Night Live sketch was exp...,PG,Comedy|Science Fiction and Fantasy,Steve Barron,Terry Turner|Tom Davis|Dan Aykroyd|Bonnie Turner,"Jul 23, 1993","Apr 17, 2001",NaN,NaN,88 minutes,Paramount Vantage
1557,1998,"Based on a novel by Richard Powell, when the l...",G,Classics|Comedy|Drama|Musical and Performing Arts,Gordon Douglas,NaN,"Jan 1, 1962","May 11, 2004",NaN,NaN,111 minutes,NaN
1558,1999,The Sandlot is a coming-of-age story about a g...,PG,Comedy|Drama|Kids and Family|Sports and Fitness,David Mickey Evans,David Mickey Evans|Robert Gunter,"Apr 1, 1993","Jan 29, 2002",NaN,NaN,101 minutes,NaN
1559,2000,"Suspended from the force, Paris cop Hubert is ...",R,Action and Adventure|Art House and Internation...,NaN,Luc Besson,"Sep 27, 2001","Feb 11, 2003",NaN,NaN,94 minutes,Columbia Pictures


In [5]:
movie_df.shape

(1560, 12)

In [6]:
movie_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1560 entries, 0 to 1559
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            1560 non-null   int64 
 1   synopsis      1498 non-null   object
 2   rating        1557 non-null   object
 3   genre         1552 non-null   object
 4   director      1361 non-null   object
 5   writer        1111 non-null   object
 6   theater_date  1201 non-null   object
 7   dvd_date      1201 non-null   object
 8   currency      340 non-null    object
 9   box_office    340 non-null    object
 10  runtime       1530 non-null   object
 11  studio        494 non-null    object
dtypes: int64(1), object(11)
memory usage: 146.4+ KB


<b> bom.movie_gross.csv</b>

In [7]:
gross_df = pd.read_csv("bom.movie_gross.csv")

In [8]:
gross_df.head()

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010
3,Inception,WB,292600000.0,535700000,2010
4,Shrek Forever After,P/DW,238700000.0,513900000,2010


In [9]:
gross_df.tail()

,title,studio,domestic_gross,foreign_gross,year
3382,The Quake,Magn.,6200.0,NaN,2018
3383,Edward II (2018 re-release),FM,4800.0,NaN,2018
3384,El Pacto,Sony,2500.0,NaN,2018
3385,The Swan,Synergetic,2400.0,NaN,2018
3386,An Actor Prepares,Grav.,1700.0,NaN,2018


In [10]:
gross_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3387 non-null   object 
 1   studio          3382 non-null   object 
 2   domestic_gross  3359 non-null   float64
 3   foreign_gross   2037 non-null   object 
 4   year            3387 non-null   int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 132.4+ KB


In [11]:
gross_df.describe()

,domestic_gross,year
count,3.359000e+03,3387.000000
mean,2.874585e+07,2013.958075
std,6.698250e+07,2.478141
min,1.000000e+02,2010.000000
25%,1.200000e+05,2012.000000
50%,1.400000e+06,2014.000000
75%,2.790000e+07,2016.000000
max,9.367000e+08,2018.000000


In [12]:
#connecting to db
conn = sqlite3.Connection('im.db')


In [13]:
#getting table names
cursor = conn.cursor()
cursor.execute("""SELECT name
    FROM sqlite_master
    WHERE type = 'table';""")
print(cursor.fetchall())

[('movie_basics',), ('directors',), ('known_for',), ('movie_akas',), ('movie_ratings',), ('persons',), ('principals',), ('writers',)]


In [14]:
pd.read_sql("""SELECT * FROM movie_basics;""",conn)

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy"
...,...,...,...,...,...,...
146139,tt9916538,Kuambil Lagi Hatiku,Kuambil Lagi Hatiku,2019,123.0,Drama
146140,tt9916622,Rodolpho Teóphilo - O Legado de um Pioneiro,Rodolpho Teóphilo - O Legado de um Pioneiro,2015,NaN,Documentary
146141,tt9916706,Dankyavar Danka,Dankyavar Danka,2013,NaN,Comedy
146142,tt9916730,6 Gunn,6 Gunn,2017,116.0,None


In [15]:
mbasics_df = pd.read_sql("""SELECT * FROM movie_basics;""",conn)

In [16]:
mbasics_df.head()

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy"


In [17]:
mbasics_df.tail()

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres
146139,tt9916538,Kuambil Lagi Hatiku,Kuambil Lagi Hatiku,2019,123.0,Drama
146140,tt9916622,Rodolpho Teóphilo - O Legado de um Pioneiro,Rodolpho Teóphilo - O Legado de um Pioneiro,2015,NaN,Documentary
146141,tt9916706,Dankyavar Danka,Dankyavar Danka,2013,NaN,Comedy
146142,tt9916730,6 Gunn,6 Gunn,2017,116.0,None
146143,tt9916754,Chico Albuquerque - Revelações,Chico Albuquerque - Revelações,2013,NaN,Documentary


In [18]:
mbasics_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146144 entries, 0 to 146143
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   movie_id         146144 non-null  object 
 1   primary_title    146144 non-null  object 
 2   original_title   146123 non-null  object 
 3   start_year       146144 non-null  int64  
 4   runtime_minutes  114405 non-null  float64
 5   genres           140736 non-null  object 
dtypes: float64(1), int64(1), object(4)
memory usage: 6.7+ MB


In [19]:
rating_df = pd.read_sql("""SELECT * FROM movie_ratings;""",conn)

In [20]:
rating_df.head()

,movie_id,averagerating,numvotes
0,tt10356526,8.3,31
1,tt10384606,8.9,559
2,tt1042974,6.4,20
3,tt1043726,4.2,50352
4,tt1060240,6.5,21


In [21]:
rating_df.tail()

,movie_id,averagerating,numvotes
73851,tt9805820,8.1,25
73852,tt9844256,7.5,24
73853,tt9851050,4.7,14
73854,tt9886934,7.0,5
73855,tt9894098,6.3,128


In [22]:
rating_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73856 entries, 0 to 73855
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   movie_id       73856 non-null  object 
 1   averagerating  73856 non-null  float64
 2   numvotes       73856 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 1.7+ MB


In [23]:
rating_df.describe()

,averagerating,numvotes
count,73856.000000,7.385600e+04
mean,6.332729,3.523662e+03
std,1.474978,3.029402e+04
min,1.000000,5.000000e+00
25%,5.500000,1.400000e+01
50%,6.500000,4.900000e+01
75%,7.400000,2.820000e+02
max,10.000000,1.841066e+06


In [51]:
#closing database
conn.close()

## Data Cleaning

### Missing Values

In [24]:
movie_df.isna().sum()

id                 0
synopsis          62
rating             3
genre              8
director         199
writer           449
theater_date     359
dvd_date         359
currency        1220
box_office      1220
runtime           30
studio          1066
dtype: int64

In [25]:
# drop those columns with more than 1000 non-null rows
movie_df = movie_df.drop(['currency', 'box_office', 'studio'],axis=1)

In [26]:
#replacing movie genre nulls with mode
genre_mode = movie_df.genre.mode()[0]
movie_df.genre.fillna(genre_mode, inplace=True)
movie_df.genre.isna().sum()

np.int64(0)

In [27]:
#replacing movie rating nulls with mode
rating_mode = movie_df.rating.mode()[0]
movie_df.rating.fillna(rating_mode, inplace=True)
movie_df.rating.isna().sum()

np.int64(0)

In [28]:
#drop the rest with nulls
movie_df.dropna(inplace=True)
movie_df.isna().sum()

id              0
synopsis        0
rating          0
genre           0
director        0
writer          0
theater_date    0
dvd_date        0
runtime         0
dtype: int64

In [29]:
gross_df.isna().sum()

title                0
studio               5
domestic_gross      28
foreign_gross     1350
year                 0
dtype: int64

In [30]:
#replacing gross for domestic and foreign with 0
gross_df.foreign_gross.fillna(0, inplace=True)
gross_df.domestic_gross.fillna(0, inplace=True)

In [31]:
#drop the rest with nulls
gross_df.dropna(inplace=True)
gross_df.isna().sum()

title             0
studio            0
domestic_gross    0
foreign_gross     0
year              0
dtype: int64

In [32]:
# converting 'foreign_gross' to float 
gross_df['foreign_gross'] = pd.to_numeric(gross_df['foreign_gross'],errors='coerce')

# calculating 'total_gross' as the sum of 'domestic_gross' and 'foreign_gross'
gross_df['total_gross'] = gross_df['domestic_gross'] + gross_df['foreign_gross']

gross_df[['domestic_gross', 'foreign_gross', 'total_gross']].head()


,domestic_gross,foreign_gross,total_gross
0,415000000.0,652000000.0,1.067000e+09
1,334200000.0,691300000.0,1.025500e+09
2,296000000.0,664300000.0,9.603000e+08
3,292600000.0,535700000.0,8.283000e+08
4,238700000.0,513900000.0,7.526000e+08


In [33]:
mbasics_df.isna().sum()

movie_id               0
primary_title          0
original_title        21
start_year             0
runtime_minutes    31739
genres              5408
dtype: int64

In [34]:
rating_df.isna().sum()

movie_id         0
averagerating    0
numvotes         0
dtype: int64

### Changing Columns

### Checking Duplicates

In [35]:
movie_df.duplicated().sum()

np.int64(0)

In [36]:
gross_df.duplicated().sum()

np.int64(0)

In [37]:
mbasics_df.duplicated().sum()

np.int64(0)

In [38]:
mbasics_df.drop_duplicates()
mbasics_df.duplicated().sum()

np.int64(0)

In [39]:
rating_df.duplicated().sum()

np.int64(0)

### Feature engineering

In [40]:
#changing runtime to int after splitting with a space to get int and 'minutes' the string
movie_df['runtime'] = movie_df['runtime'].str.split(" ").str[0]
movie_df['runtime'] = pd.to_numeric(movie_df['runtime'], errors='coerce')

# changing column name from runtime' to'runtime_in_minutes'
movie_df = movie_df.rename(columns={'runtime': 'runtime_in_minutes'})

# preview the first few rows
movie_df['runtime_in_minutes'].head()

0    104
1    108
2    116
3    128
5     95
Name: runtime_in_minutes, dtype: int64

In [41]:
# Split 'genre' into 'main_genre' and 'supporting_genre'
movie_df['main_genre'] = movie_df['genre'].str.split('|').str[0]
movie_df['supporting_genre'] = movie_df['genre'].str.split('|').apply(lambda x: '|'.join(x[1:]) if len(x) > 1 else '')

# Preview the result
movie_df[['genre', 'main_genre', 'supporting_genre']].head()


,genre,main_genre,supporting_genre
0,Action and Adventure|Classics|Drama,Action and Adventure,Classics|Drama
1,Drama|Science Fiction and Fantasy,Drama,Science Fiction and Fantasy
2,Drama|Musical and Performing Arts,Drama,Musical and Performing Arts
3,Drama|Mystery and Suspense,Drama,Mystery and Suspense
5,Drama|Kids and Family,Drama,Kids and Family


In [42]:
# Convert 'theater_date' and 'dvd_date' columns to datetime format
movie_df['theater_date'] = pd.to_datetime(movie_df['theater_date'], format='%b %d, %Y')
movie_df['dvd_date'] = pd.to_datetime(movie_df['dvd_date'], format='%b %d, %Y')

# preview the result
movie_df[['theater_date', 'dvd_date']].head()

,theater_date,dvd_date
0,1971-10-09,2001-09-25
1,2012-08-17,2013-01-01
2,1996-09-13,2000-04-18
3,1994-12-09,1997-08-27
5,2000-03-03,2000-07-11


### Saving Dataset

In [49]:
movie_df.to_csv("movie_info_clean.csv")

In [50]:
gross_df.to_csv("movie_gross_clean.csv")

In [46]:
# merging the movie_df and gross_df on 'movie_id'
movie_basics_rating_df = pd.merge(mbasics_df, rating_df, on='movie_id', how='inner')

movie_basics_rating_df.to_csv("movie_basics_rating_clean.csv")
movie_basics_rating_df.head()


,movie_id,primary_title,original_title,start_year,runtime_minutes,genres,averagerating,numvotes
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",7.0,77
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama",7.2,43
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama,6.9,4517
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama",6.1,13
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy",6.5,119


# Data Preparation


# Modeling


# Evaluation